In [1]:
import pickle
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import sklearn.metrics as sk
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import keras
from keras.datasets import cifar10

Using TensorFlow backend.


In [3]:
# Define simple model. execution starts from the bottom
def basic_mnist_model():

    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)), 
        keras.layers.BatchNormalization(),
        keras.layers.Dense(256, activation=tf.nn.relu),

        keras.layers.BatchNormalization(),
        keras.layers.Dense(256, activation=tf.nn.relu),

        keras.layers.BatchNormalization(),
        keras.layers.Dense(256, activation=tf.nn.relu),

        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    return model

In [4]:
def find_idx(arr, target):
    ans = []
    for i in range(len(arr)):
        if arr[i] == target:
            ans.append(i)
    return ans

mnist = keras.datasets.mnist
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
def get_submax(arr):
    arr = np.array(arr)
    MAX = np.max(arr)
    idx = find_idx(arr, MAX)
    arr_without_max = np.delete(arr,idx)
    return np.max(arr_without_max)
def find_statistics(Prob_Mat):
    Prob_diff = []
    MAX_Prob_Mat = []
    MAX_Prob_Mat_idx = []
    subMAX_Prob_Mat = []
    subMAX_Prob_Mat_idx = []
    for i in range(len(Prob_Mat)):
        MAX = np.max(Prob_Mat[i])
        MAX_idx = find_idx(Prob_Mat[i], MAX)[0]
        subMAX = get_submax(Prob_Mat[i])
        subMAX_idx = find_idx(Prob_Mat[i], subMAX)[0]
        prob_difference = MAX - subMAX
        Prob_diff.append(prob_difference)
        MAX_Prob_Mat.append(MAX)
        subMAX_Prob_Mat.append(subMAX)
        MAX_Prob_Mat_idx.append(MAX_idx)
        subMAX_Prob_Mat_idx.append(subMAX_idx)
    return Prob_diff,MAX_Prob_Mat,MAX_Prob_Mat_idx,subMAX_Prob_Mat,subMAX_Prob_Mat_idx
def separate_a_number(num):
    idx_train_num = find_idx(mnist_train_y,num)
    idx_train_nonum = list(set(range(len(mnist_train_y))).difference(set(idx_train_num)))
    mnist_trainx_nonum = mnist_train_x[idx_train_nonum]
    mnist_trainx_num = mnist_train_x[idx_train_num]
    mnist_trainy_nonum = mnist_train_y[idx_train_nonum]
    mnist_trainy_num = mnist_train_y[idx_train_num]
    idx_test_num = find_idx(mnist_test_y,num)
    idx_test_nonum = list(set(range(len(mnist_test_y))).difference(set(idx_test_num)))
    mnist_testx_nonum = mnist_test_x[idx_test_nonum]
    mnist_testx_num = mnist_test_x[idx_test_num]
    mnist_testy_nonum = mnist_test_y[idx_test_nonum]
    mnist_testy_num = mnist_test_y[idx_test_num]

    mnist_trainx_nonum, mnist_testx_nonum = mnist_trainx_nonum/255., mnist_testx_nonum/255.
    mnist_trainx_num, mnist_testx_num = mnist_trainx_num/255., mnist_testx_num/255.
    return mnist_trainx_nonum, mnist_trainy_nonum, mnist_testx_nonum, mnist_testy_nonum, mnist_trainx_num, mnist_trainy_num, mnist_testx_num, mnist_testy_num

In [5]:
mnist = keras.datasets.mnist
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()

In [6]:
(C_x_train, C_y_train), (C_x_test, C_y_test) = cifar10.load_data()

In [7]:
import cv2
import imutils
from skimage import exposure
X_CIFAR_grey_re_train = []
for i in range(len(C_x_train)):
    A = 0.3*C_x_train[:,:,:,0][i]+\
           0.59*C_x_train[:,:,:,1][i]+\
           0.11*C_x_train[:,:,:,2][i]
    A = exposure.rescale_intensity(A, out_range=(0, 255))
    A = imutils.resize(A, width=28)
    X_CIFAR_grey_re_train.append(A)
X_CIFAR_grey_re_train = np.array(X_CIFAR_grey_re_train)

X_CIFAR_grey_re_test = []
for i in range(len(C_x_test)):
    A = 0.3*C_x_test[:,:,:,0][i]+\
           0.59*C_x_test[:,:,:,1][i]+\
           0.11*C_x_test[:,:,:,2][i]
    A = exposure.rescale_intensity(A, out_range=(0, 255))
    A = imutils.resize(A, width=28)
    X_CIFAR_grey_re_test.append(A)
X_CIFAR_grey_re_test = np.array(X_CIFAR_grey_re_test)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
def get_the_orignal_index_after_ranking(arr, isReverse = True):
    """
    This function return the orignal index of the after the array is sorted
    inputs:
    arr || one dimension list or ndarray
    isReverse || boolean, if it is "True" the rank is decending; if it is "False" the rank is ascending 
    outputs:
    A || an arr including the orignal index before ranking
    ========================= examples =========================
    For example, arr = [4,7,2,9]
    we have the mapping relationship:
    index    value
      0        4
      1        7
      2        2
      3        9
    After sorting, say decendingly, we have:
    orignal_index     value
      3                 9
      1                 7
      0                 4
      2                 2    
    the result is for this function is [3,1,0,2].
    """
    import operator
    similarity_dict = dict(zip(list(range(len(arr))),arr))
    sorted_similarity_dict = sorted(similarity_dict.items(), reverse=isReverse, key=operator.itemgetter(1))
    A = [sorted_similarity_dict[i][0] for i in range(len(arr))]
    return A

In [9]:
# TRAIN MODEL
model = basic_mnist_model()
training_epochs = 10
learning_rate = 0.001
batch_size = 128

model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

model.fit(mnist_train_x, mnist_train_y,
      epochs=training_epochs,
      batch_size=batch_size)

test_loss, test_acc = model.evaluate(mnist_test_x, mnist_test_y)
print("Training done, test accuracy: {}".format(test_acc))

W1025 14:57:56.332714 22456 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1025 14:57:56.347421 22456 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1025 14:57:56.421222 22456 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1025 14:57:56.434270 22456 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf

Epoch 1/10
60000/60000 [==============================] - 6s 94us/step - loss: 0.2097 - acc: 0.9361
Epoch 2/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.0815 - acc: 0.9739
Epoch 3/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.0536 - acc: 0.9828
Epoch 4/10
60000/60000 [==============================] - 4s 62us/step - loss: 0.0390 - acc: 0.9873
Epoch 5/10
60000/60000 [==============================] - 4s 63us/step - loss: 0.0334 - acc: 0.9890
Epoch 6/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0290 - acc: 0.9897
Epoch 7/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0247 - acc: 0.9916
Epoch 8/10
60000/60000 [==============================] - 3s 58us/step - loss: 0.0199 - acc: 0.9933
Epoch 9/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.0220 - acc: 0.9921
Epoch 10/10
10000/10000 [==============================] - 1s 66us/step
Training done, test accuracy

In [10]:
from keras import backend as K
inp = model.input                                           # input placeholder
outputs = model.layers[7].output          # all layer outputs
functors = K.function([inp, K.learning_phase()], [outputs])

In [11]:
def minkowski_distance(x,y,n):
    if np.isinf(n):
        if n>0:
            return np.max(np.abs(x-y))
        else:
            return np.min(np.abs(x-y))
    else:
        return np.power(np.sum(np.power(np.abs(x-y),n)),1/n)
def minkowski_similarity(x,Y,n):
    arr = []
    for y in Y:
        arr.append(minkowski_distance(x,y,n))
    return np.array(arr)

def get_KNN_stats(k,testarr_one_sample, testarr_waiting_to_compare, 
                  testarr_waiting_to_compare_label, Model, similarity_method = 'cosine_similarity', minkowski_power = 2):
    """
    Inputs Example:
    k = 50
    testarr_one_sample = [mnist_train_RGB_x[0]]
    testarr_waiting_to_compare = [C_x_train[i] for i in range(5000)]
    testarr_waiting_to_compare_label = C_y_train[:5000].reshape(5000)
    Model = C_VGG_Model1.model
    
    Inputs:
    k: int, the number of the nearest neighbour
    testarr_one_sample: multi-dimensional ndarray, shape = (1,num_pixel_x,num_pixel_y,num_channel)
    testarr_waiting_to_compare_label: multi-dimensional ndarray, shape = (num_neighbour_candidate,num_pixel_x,num_pixel_y,num_channel)
    testarr_waiting_to_compare_label: one-dimensional ndarray, shape = (num_neighbour_candidate,)
    Model: keras backend model
    similarity_method: String, 'cosine_similarity', 'minkowski_similarity'. Default = 'cosine_similarity'
    minkowski_power: int, the p-value in the minkowski_distance. Only useful when similarity_method = 'minkowski_similarity'
    
    Outputs:
    similarity: one-dimensional ndarray, shape = (num_neighbour_candidate,)
    K_nearest_neighbour_orignal_label: one-dimensional ndarray, shape = (k,)
    K_nearest_neighbour: multi-dimensional ndarray, shape = (k,num_pixel_x,num_pixel_y,num_channel)
    KNN_oringal_class: dictionary, counts of the orignal class
    max_ratio_KNN_from_one_class: float, the max of the ratio of KNN are from one class
    """
    from keras import backend as K
    inp = model.input                                           # input placeholder
    outputs = model.layers[7].output          # all layer outputs
    functors = K.function([inp, K.learning_phase()], [outputs])

    # Testing
    test1 = testarr_one_sample
    layer_outs_one_sample = functors([test1, 0.])
    layer_outs_one_sample = np.array(layer_outs_one_sample)[0]

    test2 = testarr_waiting_to_compare
    layer_outs_waiting_to_compare = functors([test2, 0.])
    layer_outs_waiting_to_compare = np.array(layer_outs_waiting_to_compare)[0]
    if similarity_method == 'cosine_similarity':
        from sklearn.metrics.pairwise import cosine_similarity
        similarity = cosine_similarity(layer_outs_one_sample, layer_outs_waiting_to_compare)
        similarity = np.array(similarity[0])
    elif similarity_method == 'minkowski_similarity':
        similarity = minkowski_similarity(layer_outs_one_sample, layer_outs_waiting_to_compare, minkowski_power)
    else:
        raise Exception("invalid similarity method")
    K_nearest_neighbour_orignal_label = get_the_orignal_index_after_ranking(similarity, isReverse = False)[:k]
    K_nearest_neighbour_orignal_label = np.array(K_nearest_neighbour_orignal_label)
    K_nearest_neighbour = testarr_waiting_to_compare[K_nearest_neighbour_orignal_label]
    from collections import Counter
    KNN_oringal_class = Counter(testarr_waiting_to_compare_label[K_nearest_neighbour_orignal_label])
    max_ratio_KNN_from_one_class = max(KNN_oringal_class.values())/k
    import operator
    max_KNN_class_label = max(KNN_oringal_class.items(), key=operator.itemgetter(1))[0]
    return similarity, K_nearest_neighbour_orignal_label, K_nearest_neighbour, KNN_oringal_class, max_ratio_KNN_from_one_class, max_KNN_class_label

In [12]:
def compare_representations(testarr_one_sample, testarr_waiting_to_compare,testarr_waiting_to_compare_label, MODEL, similarity_method = 'minkowski_similarity', minkowski_power = 2):
    arr_KNN_from_same_class_ratio = []
    arr_KNN_max_class_label = []
    testarr_waiting_to_compare = testarr_waiting_to_compare[:5000]
    testarr_waiting_to_compare_label = testarr_waiting_to_compare_label[:5000].reshape(5000)
    for j in range(500):
        print("current sample: ", j)
        k = 50
        testarr_one_sample_1 = [testarr_one_sample[j]]
        result_l2 = get_KNN_stats(k,testarr_one_sample_1, testarr_waiting_to_compare, 
                  testarr_waiting_to_compare_label, MODEL, similarity_method = 'minkowski_similarity', minkowski_power = 2)
        arr_KNN_from_same_class_ratio.append(result_l2[4])
        arr_KNN_max_class_label.append(result_l2[5])

    pred_labels = MODEL.predict(testarr_one_sample[:500])
    import pandas as pd
    df = pd.DataFrame({'arr_KNN_max_class_label':arr_KNN_max_class_label,
                   'arr_KNN_from_same_class_ratio':arr_KNN_from_same_class_ratio,
                   'predicted_label':find_statistics(pred_labels)[2],
                  'predicted_prob':find_statistics(pred_labels)[1],})
    return arr_KNN_from_same_class_ratio, arr_KNN_max_class_label, df

In [13]:
result_compare = compare_representations(X_CIFAR_grey_re_train,mnist_test_x,mnist_test_y,model)

current sample:  0
current sample:  1
current sample:  2
current sample:  3
current sample:  4
current sample:  5
current sample:  6
current sample:  7
current sample:  8
current sample:  9
current sample:  10
current sample:  11
current sample:  12
current sample:  13
current sample:  14
current sample:  15
current sample:  16
current sample:  17
current sample:  18
current sample:  19
current sample:  20
current sample:  21
current sample:  22
current sample:  23
current sample:  24
current sample:  25
current sample:  26
current sample:  27
current sample:  28
current sample:  29
current sample:  30
current sample:  31
current sample:  32
current sample:  33
current sample:  34
current sample:  35
current sample:  36
current sample:  37
current sample:  38
current sample:  39
current sample:  40
current sample:  41
current sample:  42
current sample:  43
current sample:  44
current sample:  45
current sample:  46
current sample:  47
current sample:  48
current sample:  49
current sa

current sample:  396
current sample:  397
current sample:  398
current sample:  399
current sample:  400
current sample:  401
current sample:  402
current sample:  403
current sample:  404
current sample:  405
current sample:  406
current sample:  407
current sample:  408
current sample:  409
current sample:  410
current sample:  411
current sample:  412
current sample:  413
current sample:  414
current sample:  415
current sample:  416
current sample:  417
current sample:  418
current sample:  419
current sample:  420
current sample:  421
current sample:  422
current sample:  423
current sample:  424
current sample:  425
current sample:  426
current sample:  427
current sample:  428
current sample:  429
current sample:  430
current sample:  431
current sample:  432
current sample:  433
current sample:  434
current sample:  435
current sample:  436
current sample:  437
current sample:  438
current sample:  439
current sample:  440
current sample:  441
current sample:  442
current sampl

In [14]:
result_compare[2]

,arr_KNN_max_class_label,arr_KNN_from_same_class_ratio,predicted_label,predicted_prob
0,7,0.44,1,1.0
1,7,0.44,1,1.0
2,7,0.46,1,1.0
3,7,0.42,1,1.0
4,7,0.44,1,1.0
5,7,0.44,1,1.0
6,7,0.44,1,1.0
7,7,0.42,1,1.0
8,7,0.44,1,1.0
9,7,0.44,1,1.0


In [41]:
prediction = MODEL.predict(X_CIFAR_grey_re_train)

In [44]:
prediction

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)